In [5]:
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
from ipywidgets import interactive, FloatSlider

# Define anchor points
A1 = np.array([0, 2])  # Changed to (0, 2)
A2 = np.array([3, 2])  # Changed to (3, 2)

In [ ]:
# writing function to solve for the co-ordinates
def calculate_xy(T1, T2) :
    # defining / creating tensions matrix 
    T1 = np.array([T1, 0]);
    T2 = np.array([0, T2]);
    T = np.column_stack((T1,T2))
    print(f"T = {T}")
    
    # Check for singular matrix
    if np.abs(np.linalg.det(T)) < 1e-6:
        return np.nan, np.nan
        
    # Define the force vector (assuming mass = 1 kg)
    F = np.array([0, -9.81])  # Gravity force
    
    # Solve for A
    try:
        A = np.linalg.solve(T, F)
        return A[0], A[1]
    except np.linalg.LinAlgError:
        return np.nan, np.nan




In [14]:
T1 , T2 = 30.5 , 30.75

T1 = np.array([T1, 0]);
T2 = np.array([0, T2]);
T = np.column_stack((T1,T2))

T

array([[30.5 ,  0.  ],
       [ 0.  , 30.75]])

In [29]:

    
# Define the force vector (assuming mass = 1 kg)
F = np.array([0, -9.81])  # Gravity force


A = np.linalg.solve(T, F)

A

array([ 0.        , -0.31902439])

# given the tension values, it is calculating the co-ordinates 

In [87]:
import numpy as np

# Define anchor points based on your system setup
A1 = np.array([0, 2])  # Anchor point 1 (x1, y1)
A2 = np.array([3, 2])  # Anchor point 2 (x2, y2)

# Function to calculate the coordinates of the end effector given tensions T1 and T2
def calculate_coordinates(T1, T2):
    # Define the gravitational force vector (assume 1 kg mass)
    F = np.array([0, -9.81])
    
    # Calculate unit vectors pointing from the end effector towards each anchor
    def unit_vector(anchor, end_effector):
        vector = anchor - end_effector
        return vector / np.linalg.norm(vector)

    # Initial guess for the coordinates
    x_guess = 1.5  # Midpoint x between anchors as starting point
    y_guess = 1.0  # Midpoint y between anchors and ground as starting point
    
    # Numerical solver: iterate to adjust (x, y) to match given tensions
    def calculate_residuals(coords):
        x, y = coords
        end_effector = np.array([x, y])

        # Calculate unit vectors based on current guess
        u1 = unit_vector(A1, end_effector)
        u2 = unit_vector(A2, end_effector)

        # Formulate the equilibrium equations
        T1_vector = T1 * u1
        T2_vector = T2 * u2

        # Sum of tensions should counterbalance gravity
        force_balance = T1_vector + T2_vector + F

        return np.linalg.norm(force_balance)  # Residuals: magnitude should be near zero if correct

    # Use an optimization method to solve the problem
    from scipy.optimize import minimize

    # Initial guess for the solver
    initial_guess = [x_guess, y_guess]
    result = minimize(calculate_residuals, initial_guess, method='Nelder-Mead')

    # Check optimization success and return coordinates
    if result.success:
        x, y = result.x
        return x, y
    else:
        print("Error: Unable to determine coordinates with given tensions.")
        return np.nan, np.nan

# Input tensions
T1_input = float(input("Enter the tension T1 (N): "))
T2_input = float(input("Enter the tension T2 (N): "))

# Calculate coordinates
x_coord, y_coord = calculate_coordinates(T1_input, T2_input)

# Display the results
print(f"The calculated coordinates of the end effector are: X = {x_coord:.4f} m, Y = {y_coord:.4f} m.")


Enter the tension T1 (N):  18.5
Enter the tension T2 (N):  40.5


The calculated coordinates of the end effector are: X = 2.9999 m, Y = 2.0000 m.


# using kinematics , real valued things 

In [84]:
import math
import pandas as pd

# System parameters
r = 0.005  # Radius of shaft in meters
d = 0.01  # Diameter of shaft in meters
L_shaft = 0.06  # Length of shaft in meters
L_wire = 2.5654  # Length of wire in meters
G = 1.28e9  # Shear modulus of PLA in Pa

# Calculate J (polar moment of inertia)
J = (math.pi * d**4) / 32

def calculate_coordinates(T1, T2, alpha1, alpha2):
    # Calculate change in length for each wire
    def delta_l(T):
        torque = T * r
        theta = (torque * L_shaft) / (J * G)
        return r * theta

    dl1 = delta_l(T1)
    dl2 = delta_l(T2)

    # Calculate new angles
    def new_angle(alpha, dl):
        return math.acos((L_wire * math.cos(alpha)) / (L_wire + dl))

    beta1 = new_angle(alpha1, dl1)
    beta2 = new_angle(alpha2, dl2)

    # Calculate position changes
    def delta_position(alpha, beta, dl):
        dx = (L_wire + dl) * math.cos(beta) - L_wire * math.cos(alpha)
        dy = (L_wire + dl) * math.sin(beta) - L_wire * math.sin(alpha)
        return dx, dy

    dx1, dy1 = delta_position(alpha1, beta1, dl1)
    dx2, dy2 = delta_position(alpha2, beta2, dl2)

    # Combine the effects of both wires
    # Assuming the wires are symmetrically placed
    x = (dx1 - dx2) / 2
    y = (dy1 + dy2) / 2

    return x, y

def process_tension_pairs(tension_pairs):
    results = []
    alpha1 = math.radians(30)  # 30 degrees from horizontal
    alpha2 = math.radians(150)  # 150 degrees from horizontal

    for T1, T2 in tension_pairs:
        x, y = calculate_coordinates(T1, T2, alpha1, alpha2)
        results.append({'T1': T1, 'T2': T2, 'X': x, 'Y': y})
    
    return pd.DataFrame(results)

# Example usage:
tension_pairs = [
    (18.5, 40.5),
    # Add more pairs as needed
]

result_df = process_tension_pairs(tension_pairs)
print(result_df)

# To save results to a CSV file:
# result_df.to_csv('tension_coordinates_results.csv', index=False)

     T1    T2    X        Y
0  18.5  40.5  0.0  0.00007


# TRY